In [104]:
import pandas as pd

In [105]:
cons_info = pd.read_csv("https://als-hiring.s3.amazonaws.com/fake_data/2020-07-01_17%3A11%3A00/cons.csv")
cons_email = pd.read_csv("https://als-hiring.s3.amazonaws.com/fake_data/2020-07-01_17%3A11%3A00/cons_email.csv")
cons_status = pd.read_csv("https://als-hiring.s3.amazonaws.com/fake_data/2020-07-01_17%3A11%3A00/cons_email_chapter_subscription.csv")
cons_status = cons_status[cons_status["chapter_id"]==1]

In [106]:
merged1 = pd.merge(cons_email, cons_status, on="cons_email_id", how="inner")
merged2 = pd.merge(merged1, cons_info, on="cons_id", how="inner")

In [107]:
people = merged2[["email", "source", "isunsub", "create_dt_x", "modified_dt_x"]]
people = people.rename(columns={"source":"code", "isunsub":"is_unsub", "create_dt_x":"created_dt", "modified_dt_x":"updated_dt"})
people = people.astype({"email":str, "code":str})
people["created_dt"] = pd.to_datetime(people["created_dt"])
people["updated_dt"] = pd.to_datetime(people["updated_dt"])
people.is_unsub.replace({1:True, 0:False}, inplace=True)
people.to_csv("people.csv")

In [108]:
date_grouped = people.groupby(pd.Grouper(key="created_dt", axis=0, freq="1D", sort=True)).sum()
acquisition_facts = pd.DataFrame(date_grouped)
acquisition_facts = acquisition_facts.reset_index()
acquisition_facts = acquisition_facts.rename(columns={"created_dt":"acquisition_date","is_unsub":"acquisitions"})
acquisition_facts["acquisition_date"] = pd.to_datetime(acquisition_facts["acquisition_date"]).dt.date
acquisition_facts.to_csv("acquisition_facts.csv")